# Import Ocelot output to Brightway2

## Warning: This was developed during the sumer school, it still has bugs!

Import basic things from both systems

In [1]:
from ocelot import *
from brightway2 import *

Run the Ocelot system model. `data` is the fully linked version of the database.

In [2]:
# Adapt this file path as needed
fp, data = system_model("/Users/cmutel/Documents/LCA Documents/Ecoinvent/3.2/unlinked/")

Starting Ocelot model run
Using cached ecospold2 data
Opening log file at: /Users/cmutel/Library/Application Support/Ocelot/model-runs/299ec857aaa64278acb92a626d491bd0/report.log.json
Applying transformation variable_names_are_unique
Applying transformation ensure_markets_only_have_one_reference_product
Applying transformation ensure_markets_dont_consume_their_ref_product
Applying transformation fix_specific_ecoinvent_issues
Applying transformation replace_implicit_references
Applying transformation fix_known_bad_formula_strings
Applying transformation lowercase_all_parameters
Applying transformation fix_math_formulas
Applying transformation replace_reserved_words
Applying transformation delete_unparsable_formulas
Applying transformation ensure_production_exchanges_have_production_volume
Applying transformation add_pv_to_allocatable_byproducts
Applying transformation create_pv_parameters
Applying transformation remove_consequential_exchanges
Applying transformation drop_rp_activity_lin

NameError
   5.36e-11* cement+silica_fume*0+ ggbfs*0
                     ^^^
name 'silica_fume' is not defined
NameError
   5.36e-11* cement+silica_fume*0+ ggbfs*0
                     ^^^
name 'silica_fume' is not defined


Applying transformation combined_production_with_byproducts
Applying transformation delete_allocation_method
Applying transformation drop_rp_activity_links
Applying transformation label_reference_product
Applying transformation delete_allowed_zero_pv_market_datsets
Applying transformation assign_fake_pv_to_confidential_datasets
Applying transformation relabel_global_to_row
Applying transformation add_unique_codes
Applying transformation actualize_activity_links
Applying transformation add_recycled_content_suppliers_to_markets
Applying transformation add_suppliers_to_markets
Applying transformation update_market_production_volumes
Applying transformation add_suppliers_to_markets
Applying transformation update_market_production_volumes
Applying transformation allocate_suppliers
Skipping zero total PV with multiple inputs:
	market for waste newspaper/waste newspaper (GLO, 3 suppliers)
Applying transformation link_consumers_to_regional_markets
Applying transformation link_consumers_to_recy

Brightway2 splits work into [projects](https://docs.brightwaylca.org/intro.html#projects). 

We create a new project for this database.

In [3]:
projects.set_current("ocelot test for summer school")

Create base data: LCIA methods and biosphere database.

In [4]:
bw2setup()

Creating default biosphere



Writing activities to SQLite3 database:
0%                          100%
[###                           ] | ETA: 00:00:01

Applying strategy: normalize_units
Applying strategy: drop_unspecified_subcategories
Applied 2 strategies in 0.01 seconds


[##############################] | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 09/08/2016 16:02:46
  Finished: 09/08/2016 16:02:47
  Total time elapsed: 00:00:00
  CPU %: 92.30
  Memory %: 26.26
Created database: biosphere3
Creating default LCIA methods

Applying strategy: normalize_units
Applying strategy: set_biosphere_type
Applying strategy: drop_unspecified_subcategories
Applying strategy: link_iterable_by_fields
Applied 4 strategies in 1.74 seconds
Wrote 665 LCIA methods with 169551 characterization factors
Creating core data migrations



Import the Brightway2 importer/conversion utility.

In [6]:
from ocelot.io.brightway2 import *

Create a new Brightway2 database

In [7]:
new_db_name = "ecoinvent 3.2 cutoff (ocelot)"
db = import_into_brightway2(data, new_db_name)

Creating database `ecoinvent 3.2 cutoff (ocelot)` in project `ocelot test for summer school`
Applying strategy: normalize_units
Applying strategy: drop_unspecified_subcategories
Applying strategy: strip_biosphere_exc_locations
Applied 3 strategies in 0.50 seconds
Applying strategy: link_iterable_by_fields


Writing activities to SQLite3 database:
0%                          100%
[                              ]

13245 datasets
391343 exchanges
0 unlinked exchanges
  
Writing database


[##############################] | ETA: 00:00:00
Total time elapsed: 00:01:19


Title: Writing activities to SQLite3 database:
  Started: 09/08/2016 16:05:18
  Finished: 09/08/2016 16:06:37
  Total time elapsed: 00:01:19
  CPU %: 89.90
  Memory %: 25.40
Created database: ecoinvent 3.2 cutoff (ocelot)


Brightway2 SQLiteBackend: ecoinvent 3.2 cutoff (ocelot)

Check to see how many processes are in our new database

In [8]:
len(db)

13245


In [ ]:
Find an LCIA method

In [10]:
sorted([method for method in methods if "IPCC" in str(method)])  # Could also check for "ReCiPe"

[('IPCC 2007', 'GWP', '500 years'),
 ('IPCC 2013', 'GWP', '20 years'),
 ('IPCC 2007', 'GWP', '100 years'),
 ('IPCC 2013', 'GWP', '500 years'),
 ('IPCC 2013', 'GWP', '100 years'),
 ('IPCC 2001', 'climate change', 'GWP 100a'),
 ('IPCC 2001', 'climate change', 'GWP 500a'),
 ('IPCC 2001', 'climate change', 'GWP 20a'),
 ('IPCC 2007', 'GWP', '20 years')]

Create a new LCA object. Pick a functional unit at random.

In [11]:
lca = LCA({db.random(): 1}, method=('IPCC 2013', 'GWP', '100 years'))

In [15]:
lca.demand

{'sodium phosphate production' (kilogram, RoW, None): 1}

Factorizing the database makes subsequent calculations quicker.

In [12]:
lca.lci(factorize=True)

Current scores are wrong!

In [14]:
lca.lcia()
lca.score

-3161261260216.592

In [ ]:
for ds in db:
    lca.redo_lcia({ds: 1})
    print(lca.score)